# Section 1

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [4]:
!pip install BeautifulSoup4
!pip install requests

In [5]:
from bs4 import BeautifulSoup

Webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(Webpage.text, 'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###  Ignore cells with a borough that is Not assigned.

In [6]:
toronto_df.shape

(288, 3)

In [7]:
toronto_df.Borough.head()

0        Not assigned
1        Not assigned
2          North York
3          North York
4    Downtown Toronto
Name: Borough, dtype: object

In [8]:
Torontoclean_df = toronto_df[toronto_df.Borough != 'Not assigned']
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Index are reset

In [9]:
Torontoclean_df = Torontoclean_df.reset_index(drop=True)
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [10]:
Torontoclean_df.shape

(211, 3)

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
Torontoclean_df[Torontoclean_df.Neighbourhood == 'Not assigned'].count()

Postcode         1
Borough          1
Neighbourhood    1
dtype: int64

In [12]:
Torontoclean_df['Neighbourhood'].replace("Not assigned", Torontoclean_df["Borough"],inplace=True)
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [13]:
Torontoclean_df[Torontoclean_df.Neighbourhood == 'Not assigned'].count()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

In [14]:
Torontoclean_df.groupby("Postcode")["Neighbourhood"].sum().count()

103

### We here try to manage to use comma between different neighbourhood

In [15]:
cluster_toronto = Torontoclean_df.groupby("Postcode").Neighbourhood.agg([('Neighbourhood', ', '.join)])

In [16]:
cluster_toronto.head()

,Neighbourhood
Postcode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae


In [17]:
cluster_toronto['Borough'] = Torontoclean_df.groupby("Postcode").Borough.agg([('Borough', ', '.join)])
cluster_toronto.head()

,Neighbourhood,Borough
Postcode,,
M1B,"Rouge, Malvern","Scarborough, Scarborough"
M1C,"Highland Creek, Rouge Hill, Port Union","Scarborough, Scarborough, Scarborough"
M1E,"Guildwood, Morningside, West Hill","Scarborough, Scarborough, Scarborough"
M1G,Woburn,Scarborough
M1H,Cedarbrae,Scarborough


In [18]:
new = cluster_toronto["Borough"].str.split(',', n = 1, expand = True)

In [19]:
cluster_toronto["Borough"] = new[0]

In [20]:
columnsTitles=["Borough","Neighbourhood"]
cluster_toronto=cluster_toronto.reindex(columns=columnsTitles)

In [21]:
cluster_toronto.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [22]:
cluster_toronto.shape

(103, 2)

# Section 2
## Reading latitude and langitude of neighbours

In [36]:
lat_long_df = pd.read_csv("Geospatial_Coordinates.csv")
lat_long_df.columns = ['Postcode', 'Latitude', 'Longitude']
lat_long_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### The new data is sorted and postcode is set as index

In [37]:
lat_long_df.sort_values(by=['Postcode'], inplace=True)
lat_long_df.set_index('Postcode', inplace=True)
lat_long_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


### The previous data is now also sorted

In [38]:
cluster_toronto.sort_values(by=['Postcode'], inplace=True)
cluster_toronto.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [41]:
cluster_toronto['Latitude'] = lat_long_df['Latitude']
cluster_toronto['Longitude'] = lat_long_df['Longitude']

In [43]:
cluster_toronto.head(11)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
